In [ ]:
#pip install gspread
#pip install gspread-dataframe
#pip install google
#pip install pydrive
#%pip install selenium
#pip install pydrive2

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import date
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import numpy as np
import os
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

In [ ]:
scopes = ['https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file('your path go here')
gc = gspread.authorize(credentials)
gauth = GoogleAuth()
drive = GoogleDrive(gauth)
# open a google sheet
gs = gc.open_by_url('https://docs.google.com/spreadsheets/d/1b06QOCAlYdmRKyW2sp1wGI-I_nU1iilteJPS5lR094c/edit#gid=1100797668')
# select a work sheet from its name
worksheet1 = gs.worksheet('Linkedin_Jobdata')

In [ ]:
#Website
url =' https://ca.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Canada&locationId=&geoId=101174742&f_TPR=r86400&position=1&pageNum=0'
#Get the url
s = Service('Your path go here')
driver = webdriver.Chrome(service=s)
driver.get(url)
driver.maximize_window()
time.sleep(15)

In [ ]:
#Get scroll Height
CLICK_PAUSE_TIME = 10
last_height = driver.execute_script("return document.body.scrollHeight")

In [ ]:
#Scrolling the page til seeing see more
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    # Wait to load page - Do not open tab as it will affect the scrollingHeight
    time.sleep(5)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    print= new_height
    if new_height == last_height:
    break
    last_height = new_height

In [ ]:
#Clicking see more til no more new input
while True:
try:
element = WebDriverWait(driver,CLICK_PAUSE_TIME).until(EC.element_to_be_clicka
driver.execute_script("arguments[0].click();", element)
except Exception as e:
pass
time.sleep(5)
new_height = driver.execute_script("return document.body.scrollHeight")
if new_height == last_height:
break
last_height = new_height

In [ ]:
#find all elements with posting id using xpath putting everthing to list (if XPATH li[
job_lists = driver.find_elements(By.XPATH,"//*[@id='main-content']/section[2]/ul/li")
len(job_lists)

In [ ]:
#Retrieve the job ID, title, company_name, location, last_update, job_link information
#Procedure we find the Parent element (usually through XPATH link), retrieve a list, then looping through to find all the information
#through other locator (CSS_SELECTOR,CLASS_NAME)
job_title_list = []
company_name_list = []
location_list =[]
date_time_list=[]
job_link_list = []

for job in job_lists:
    job_title = None
    company_name = None
    date_time = None
    job_link = None
    
    try:
        job_title = job.find_element(By.CSS_SELECTOR,'h3').get_attribute('innerText')
        company_name = job.find_element(By.CSS_SELECTOR,'h4').get_attribute('innerText
        location = job.find_element(By.CLASS_NAME,'job-search-card__location').get_att
        date_time = job.find_element(By.CSS_SELECTOR,'time').get_attribute("datetime")
        job_link = job.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
    except Exeception as e:
        pass
    job_title_list.append(job_title)
    company_name_list.append(company_name)
    location_list.append(location)
    date_time_list.append(date_time)
    job_link_list.append(job_link)

In [ ]:
#Create Dataframe
job_data = pd.DataFrame({'ID': job_id_list,
'Last_Posting_Date': date_time_list,
'Company': company_name_list,
'Title': job_title_list,
'Location': location_list,
'Description': jd_list,
'Level': seniority_list,
'Type': emp_type_list,
'Function': job_func_list,
'Industry': industries_list,
'Link': job_link_list
})                         

In [ ]:
#Clearing job with duplicate id
# cleaning description column
job_data['Description'] = job_data['Description'].str.replace("Show more","")
job_data= job_data.replace('\n'," ", regex = True)

In [ ]:
#moving some full time value from seniority to type
for x in range(len(job_data)):
if job_data['Level'][x] in ["Full-time","Contract","Other","Part-time","Temporary")
job_data['Type'][x] = job_data['Level'][x]
job_data['Level'][x] = None

In [ ]:
#extract to csv - write header to
output_path = 'your path'
#write more data to the existing csv.
job_data.to_csv(output_path, mode ='a', index = False, header = not os.path.exists(output_path)) #if the file exist then does not include header, if not include header

In [ ]:
#Write Data to GGSheet
# write to dataframe - for first time inputting to sheet
#set_with_dataframe(worksheet=worksheet1, dataframe=job_data, include_index=False,
#include_column_header=True, resize=True)
jobdata_value_list = job_data.values.tolist()
gs.values_append('Linkedin_Jobdata', {'valueInputOption': 'USER_ENTERED'}, {'values': jobdata_value_list})